In [1]:
# 02_frontend.ipynb
# "Mini front end" for running backtests with simple inputs

from datetime import date
from pathlib import Path
import os

import pandas as pd

from backtest_engine import BacktestEngine, PostgresDataProvider, run_batch
from backtest_engine.config import (
    BacktestConfig,
    StudyWindow,
    UniverseConfig,
    SignalConfig,
    SelectionConfig,
    RebalanceConfig,
    FeeConfig,
)

pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.width = 160

print("✅ Imports OK")


✅ Imports OK


In [2]:
# OPTIONAL: Set PG* env vars here if they are missing.
# Comment out or edit as you like.

defaults = {
    "PGHOST": "localhost",
    "PGPORT": "5433",
    "PGDATABASE": "kairo_production",
    "PGUSER": "sanjay_readonly",
    "PGPASSWORD": "Piper112358!",
}

for key, value in defaults.items():
    if not os.getenv(key):
        os.environ[key] = value

print("PGHOST =", os.getenv("PGHOST"))
print("PGPORT =", os.getenv("PGPORT"))
print("PGDATABASE =", os.getenv("PGDATABASE"))
print("PGUSER =", os.getenv("PGUSER"))
print("✅ PG* env vars are set (assuming SSH tunnel is running)")


PGHOST = localhost
PGPORT = 5433
PGDATABASE = kairo_production
PGUSER = sanjay_readonly
✅ PG* env vars are set (assuming SSH tunnel is running)


In [3]:
from backtest_engine.db import run_sql

provider = PostgresDataProvider()

try:
    df_now = run_sql("SELECT now() AT TIME ZONE 'Asia/Kolkata' AS ist_now;")
    display(df_now)
    print("✅ DB connected and responding")
except Exception as e:
    print("❌ DB connectivity issue:", repr(e))
    print("Check SSH tunnel + PG* environment variables.")


,ist_now
0,2025-11-28 11:39:21.226028


✅ DB connected and responding


In [4]:
def make_config(
    name: str,
    start: date,
    end: date,
    rebalance_freq: str = "12M",
    universe_preset: str = "equity_active_direct",
    signal_name: str = "rank_12m_category",
    signal_direction: str = "asc",
    selection_mode: str = "top_n",
    top_n: int = 15,
    min_funds: int = 10,
    weight_scheme: str = "equal",
    fee_bps: float = 0.0,
    apply_fee: bool = False,
) -> BacktestConfig:
    """Convenience builder for BacktestConfig.

    This is where we centralise sensible defaults for the 'frontend'.
    """
    return BacktestConfig(
        name=name,
        study_window=StudyWindow(start=start, end=end),
        universe=UniverseConfig(preset=universe_preset),
        signal=SignalConfig(
            name=signal_name,
            direction=signal_direction,  # "asc" -> lower score is better, for ranks
        ),
        selection=SelectionConfig(
            mode=selection_mode,        # "top_n" or "all"
            top_n=top_n,
            min_funds=min_funds,
            weight_scheme=weight_scheme,
        ),
        rebalance=RebalanceConfig(
            frequency=rebalance_freq,   # "NONE", "3M", "6M", "12M", "18M", "24M"
        ),
        fees=FeeConfig(
            apply=apply_fee,
            annual_bps=fee_bps,
        ),
    )


In [5]:
# === SINGLE RUN INPUTS ===
# Edit these values and then run this cell + the next one.

RUN_NAME = "mf-10y-annual-top15-net1%"   # used in filenames, folder names

START_DATE = date(2014, 1, 1)
END_DATE   = date(2024, 1, 1)

UNIVERSE_PRESET = "equity_active_direct"

SIGNAL_NAME       = "rank_12m_category"
SIGNAL_DIRECTION  = "asc"    # "asc" for rank columns (1 = best), "desc" for perf columns

SELECTION_MODE = "top_n"     # "top_n" or "all"
TOP_N          = 15
MIN_FUNDS      = 10
WEIGHT_SCHEME  = "equal"     # only "equal" is implemented

REBALANCE_FREQ = "12M"       # "NONE", "3M", "6M", "12M", "18M", "24M"

APPLY_FEE = True
FEE_BPS   = 100.0            # 100 bps = 1% p.a.

# Where to save this run's outputs
SINGLE_RUN_OUTDIR = Path("outputs/single_runs")

# Output detail level: "light", "standard", "full"
SAVE_LEVEL = "standard"

print("Configured single run:", RUN_NAME)


Configured single run: mf-10y-annual-top15-net1%


In [6]:
# Run a single backtest based on the form above

engine = BacktestEngine(provider)

single_config = make_config(
    name=RUN_NAME,
    start=START_DATE,
    end=END_DATE,
    rebalance_freq=REBALANCE_FREQ,
    universe_preset=UNIVERSE_PRESET,
    signal_name=SIGNAL_NAME,
    signal_direction=SIGNAL_DIRECTION,
    selection_mode=SELECTION_MODE,
    top_n=TOP_N,
    min_funds=MIN_FUNDS,
    weight_scheme=WEIGHT_SCHEME,
    fee_bps=FEE_BPS,
    apply_fee=APPLY_FEE,
)

print("Running config:")
display(single_config)

result = engine.run(single_config)
print("✅ Backtest completed")

display(result.summary)


Running config:


BacktestConfig(name='mf-10y-annual-top15-net1%', study_window=StudyWindow(start=datetime.date(2014, 1, 1), end=datetime.date(2024, 1, 1)), universe=UniverseConfig(preset='equity_active_direct', filters={}), signal=SignalConfig(name='rank_12m_category', source='performance_ranking', lookback_months=12, direction='asc', rank_scope='category', tie_breaker=None, expression=None, filter_expression=None), selection=SelectionConfig(mode='top_n', top_n=15, min_funds=10, weight_scheme='equal'), rebalance=RebalanceConfig(frequency='12M', anchor='month_end'), fees=FeeConfig(apply=True, annual_bps=100.0, apply_frequency='daily'), tax=TaxConfig(apply=False, stcg_rate=0.15, ltcg_rate=0.1, ltcg_holding_days=365), metadata={})

✅ Backtest completed


,run_id,name,start_date,end_date,num_periods,gross_return,gross_cagr,net_return,net_cagr,benchmark_return,benchmark_cagr,alpha_return,alpha_cagr,net_alpha_return,net_alpha_cagr,fees_applied,fees_annual_bps
0,mf-10y-annual-top15-net1%,mf-10y-annual-top15-net1%,2014-01-01,2024-01-01,10,4.806077,0.192193,4.250623,0.180271,2.872259,0.144891,1.933818,0.047301,1.378364,0.035379,True,100.0


In [7]:
# Period-level summary (per rebalance period)

cols_periods = [
    "period_no",
    "start_date",
    "end_date",
    "period_days",
    "num_funds",
    "gross_return",
    "net_return",
    "benchmark_return",
    "alpha_return",
    "net_alpha_return",
]

display(result.portfolio_periods[cols_periods].head(20))

# Optional: holdings sample (first 20 rows)

cols_holdings = [
    "run_id",
    "period_no",
    "rebalance_date",
    "schemecode",
    "scheme_name",
    "weight",
    "fund_return",
    "period_gross_return",
    "period_net_return",
]

display(result.holdings[cols_holdings].head(20))


,period_no,start_date,end_date,period_days,num_funds,gross_return,net_return,benchmark_return,alpha_return,net_alpha_return
0,1,2014-01-01,2015-01-01,365,15,0.510397,0.495293,0.378938,0.131459,0.116355
1,2,2015-01-01,2016-01-01,365,15,0.046460,0.035995,-0.004782,0.051242,0.040777
2,3,2016-01-01,2017-01-01,366,15,0.038411,0.027999,0.033930,0.004482,-0.005931
3,4,2017-01-01,2018-01-01,365,15,0.376957,0.363188,0.347305,0.029653,0.015883
4,5,2018-01-01,2019-01-01,365,15,-0.086565,-0.095700,-0.025078,-0.061487,-0.070621
5,6,2019-01-01,2020-01-01,365,15,0.088254,0.077371,0.075088,0.013166,0.002283
6,7,2020-01-01,2021-01-01,366,15,0.202399,0.190343,0.170531,0.031869,0.019812
7,8,2021-01-01,2022-01-01,365,15,0.475905,0.461146,0.295585,0.180320,0.165561
8,9,2022-01-01,2023-01-01,365,15,0.050889,0.040381,0.016064,0.034825,0.024316
9,10,2023-01-01,2024-01-01,365,15,0.385821,0.371963,0.254183,0.131638,0.117780


,run_id,period_no,rebalance_date,schemecode,scheme_name,weight,fund_return,period_gross_return,period_net_return
0,mf-10y-annual-top15-net1%,1,2014-01-01,18512,ICICI Pru Technology Fund(G)-Direct Plan,0.066667,0.281517,0.510397,0.495293
1,mf-10y-annual-top15-net1%,1,2014-01-01,19008,SBI Midcap Fund(G)-Direct Plan,0.066667,0.732402,0.510397,0.495293
2,mf-10y-annual-top15-net1%,1,2014-01-01,18465,HSBC Flexi Cap Fund(G)-Direct Plan,0.066667,0.560412,0.510397,0.495293
3,mf-10y-annual-top15-net1%,1,2014-01-01,18192,Axis ELSS Tax Saver Fund(G)-Direct Plan,0.066667,0.693876,0.510397,0.495293
4,mf-10y-annual-top15-net1%,1,2014-01-01,18146,Franklin India Focused Equity Fund(G)-Direct Plan,0.066667,0.806433,0.510397,0.495293
5,mf-10y-annual-top15-net1%,1,2014-01-01,18924,Invesco India Large & Mid Cap Fund(G)-Direct Plan,0.066667,0.457385,0.510397,0.495293
6,mf-10y-annual-top15-net1%,1,2014-01-01,18186,Axis Large Cap Fund(G)-Direct Plan,0.066667,0.426407,0.510397,0.495293
7,mf-10y-annual-top15-net1%,1,2014-01-01,18106,HDFC Capital Builder Value Fund(G)-Direct Plan,0.066667,0.528956,0.510397,0.495293
8,mf-10y-annual-top15-net1%,1,2014-01-01,18635,Kotak Contra Fund(G)-Direct Plan,0.066667,0.402472,0.510397,0.495293
9,mf-10y-annual-top15-net1%,1,2014-01-01,18134,Franklin India Small Cap Fund(G)-Direct Plan,0.066667,0.910110,0.510397,0.495293


In [8]:
SINGLE_RUN_OUTDIR.mkdir(parents=True, exist_ok=True)

paths_single = result.save(SINGLE_RUN_OUTDIR, level=SAVE_LEVEL)
print("Saved files:")
for key, path in paths_single.items():
    print(f"  {key}: {path}")


Saved files:
  summary: outputs/single_runs/mf-10y-annual-top15-net1%.summary.csv
  portfolio_periods: outputs/single_runs/mf-10y-annual-top15-net1%.periods.csv


In [9]:
# === BATCH RUN EXAMPLE ===
# Define multiple configs (variants) and run them all in one go.

BATCH_OUTDIR = Path("batch_outputs/frontend_examples")

base_window = StudyWindow(start=date(2014, 1, 1), end=date(2024, 1, 1))

batch_configs = [
    make_config(
        name="mf-10y-annual-top15-net1%",
        start=base_window.start,
        end=base_window.end,
        rebalance_freq="12M",
        top_n=15,
        min_funds=10,
        fee_bps=100.0,
        apply_fee=True,
    ),
    make_config(
        name="mf-10y-annual-top30-net1%",
        start=base_window.start,
        end=base_window.end,
        rebalance_freq="12M",
        top_n=30,
        min_funds=20,
        fee_bps=100.0,
        apply_fee=True,
    ),
    make_config(
        name="mf-10y-semiannual-top15-net1%",
        start=base_window.start,
        end=base_window.end,
        rebalance_freq="6M",
        top_n=15,
        min_funds=10,
        fee_bps=100.0,
        apply_fee=True,
    ),
]

print("Batch configs:")
for cfg in batch_configs:
    print(" -", cfg.name)


Batch configs:
 - mf-10y-annual-top15-net1%
 - mf-10y-annual-top30-net1%
 - mf-10y-semiannual-top15-net1%


In [10]:
batch_summary = run_batch(
    configs=batch_configs,
    data_provider=provider,
    out_dir=BATCH_OUTDIR,
    level="standard",  # or "light" / "full"
)

print("✅ Batch completed")
display(batch_summary)


✅ Batch completed


,run_id,name,start_date,end_date,num_periods,gross_return,gross_cagr,net_return,net_cagr,benchmark_return,benchmark_cagr,alpha_return,alpha_cagr,net_alpha_return,net_alpha_cagr,fees_applied,fees_annual_bps,output_dir,output_files
0,mf-10y-annual-top15-net1%,mf-10y-annual-top15-net1%,2014-01-01,2024-01-01,10,4.806077,0.192193,4.250623,0.180271,2.872259,0.144891,1.933818,0.047301,1.378364,0.035379,True,100.0,batch_outputs/frontend_examples/mf-10y-annual-...,batch_outputs/frontend_examples/mf-10y-annual-...
1,mf-10y-annual-top30-net1%,mf-10y-annual-top30-net1%,2014-01-01,2024-01-01,10,4.520314,0.186194,3.992198,0.174332,2.872259,0.144891,1.648055,0.041303,1.119939,0.029441,True,100.0,batch_outputs/frontend_examples/mf-10y-annual-...,batch_outputs/frontend_examples/mf-10y-annual-...
2,mf-10y-semiannual-top15-net1%,mf-10y-semiannual-top15-net1%,2014-01-01,2024-01-01,20,4.925563,0.194622,4.358678,0.182676,2.832886,0.143722,2.092677,0.050900,1.525792,0.038954,True,100.0,batch_outputs/frontend_examples/mf-10y-semiann...,batch_outputs/frontend_examples/mf-10y-semiann...
